# Endemo 2 - Useful Energy Demand Model Rework

## Industry Sector



### Import Packages

In [1]:
import pathlib

import xlrd as x
import pandas as pd
from pathlib import Path
import os

### Import Custom Classes

In [2]:
from control_parameters import ControlParameters
from industry_sector import Industry
from household_sector import Household
from transport_sector import Transport
from commercial_trade_services_sector import CommercialTradeServices
from countries import Countries

C:\Users\Chris\Documents\Master_Semester_1\IDP\endemo2\ControlParameters.py:26: SyntaxWarning: "is" with a literal. Did you mean "=="?
  return [sector for (sector, isActive) in self._sectors_active_values.items() if isActive is 1]


### Associate strings with code

SECTOR_NAME_TO_OBJECT_MAP: necessary to connect the activated sectors in Set_and_Control_Parameters.xlsl with the code execution

In [3]:
SECTOR_NAME_TO_OBJECT_MAP = dict({"industry":Industry(), "households":Household(), "transport":Transport(), "commertial, trade, services":CommercialTradeServices()})




### Set Input Paths

In [4]:
# change path to point to your endemo2 folder

super_path = Path(os.path.abspath(''))
input_path = super_path / 'input'
print("Input folder at " + str(input_path))

general_path = input_path / 'general'
industry_path = input_path / 'industry'


Input folder at C:\Users\Chris\Documents\Master_Semester_1\IDP\endemo2\input


### Read control parameters

In [5]:
set_and_control_parameters = pd.read_excel(input_path / 'Set_and_Control_Parameters.xlsx')
ctrl = ControlParameters(set_and_control_parameters)
print(ctrl)

sectors_active_values: {'industry': 1, 'households': 1, 'transport': 1, 'commertial, trade, services': 1}
parameter_values: {'Forecast year': 2050, 'Population forecast': 1, 'Timeseries forecast': 1, 'NUTS2 geographical resolution': 1, 'World calculation': 0, 'NUTS2 classification': 2016, 'Graphical output': 1}


### Read available countries

In [9]:
abbreviations = pd.read_excel(general_path / 'Abbreviations.xlsx')
countries = Countries(abbreviations)
print(countries)

{'Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antarctica',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bonaire, Sint Eustatius and Saba',
 'Bosnia and Herzegovina',
 'Botswana',
 'Bouvet Island',
 'Brazil',
 'British Indian Ocean Territory',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Christmas Island',
 'Cocos (Keeling) Islands',
 'Colombia',
 'Comoros',
 'Congo',
 'Cook Islands',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Curaçao',
 'Cyprus',
 'Czechia',
 "Côte d'Ivoire",
 'DR Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',

### Read production data

In [6]:

products = {'Aluminium', 'Ammonia', 'Aromate', 'Cement', 'Chlorin', 'Cooper', 'Ethylene', 'Glass', 'Paper', 'Propylene', 'Steel'}

for product in products:
    file_name = product + "_Production.xlsx"
    # todo: read file and create product object (product class needs to be implemented first